In [3]:
import os

# print(os.getenv("OPENAI_API_KEY"))

In [6]:
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": "What's today's date?"}],
)

response

ChatCompletion(id='chatcmpl-C8LFkavpsREYmGtuLgMky6N2yjJ2O', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Today is August 25, 2025.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1756104356, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=275, prompt_tokens=10, total_tokens=285, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=256, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Memory


In [11]:
import openai

client = openai.OpenAI()

messages = []

In [3]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
    )

    # memory에 추가
    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print(f"AI: {message}")

In [ ]:
while True:
    message = input("Input for LLM")

    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: I have a dog. Answer briefly.
AI: Nice! What would you like help with—training, health, diet, or behavior? I can give quick, practical tips.
User: What kind of animal do I have?
AI: A dog. Want tips on training, health, or care?
User: 
AI: If you’d like, tell me what you want help with—training, health, feeding, or behavior. Any details about breed, age, or goals will help me tailor tips.


### Tools


In [12]:
# Tool Schema
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [ ]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        tools=TOOLS,  # Adding Tools
    )

    print(response)

    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print(f"AI: {message}")


while True:
    message = input("Input for LLM")

    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: My name is JM. Answer briefly.
ChatCompletion(id='chatcmpl-C8diru9CG43BmnsGr8nMNfyh1UrMG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, JM. How can I help today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1756175353, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=278, prompt_tokens=148, total_tokens=426, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=256, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
AI: Nice to meet you, JM. How can I help today?
User: Weather in Jeju
ChatCompletion(id='chatcmpl-C8dj6rnSbrdzmnwuW95FYyym3sYhs', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMe

In [ ]:
# AI: None ??
ChatCompletion(
    id="chatcmpl-C8dj6rnSbrdzmnwuW95FYyym3sYhs",
    choices=[
        Choice(
            finish_reason="tool_calls",
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,  # None
                refusal=None,
                role="assistant",
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=[  # Tool Calls
                    ChatCompletionMessageFunctionToolCall(
                        id="call_kBmPM97Q6SRqwNMekqcXgOs7",
                        function=Function(
                            arguments='{"city":"Jeju"}', name="get_weather"
                        ),
                        type="function",
                    )
                ],
            ),
        )
    ],
    created=1756175368,
    model="gpt-5-nano-2025-08-07",
    object="chat.completion",
    ...
)

In [13]:
def get_weather(city: str):
    return "00 degrees celcius."


FUNCTION_MAP = {"get_weather": get_weather}

In [14]:
from openai.types.chat import ChatCompletionMessage
import json


def process_ai_response(message: ChatCompletionMessage):
    """
    message from ai.
    run if the tool_call in messages.
    add messages to memory and run tools.
    and call_ai() with new messages (+ tool_request, tool_result)
    """

    if message.tool_calls:

        # to memory(request)
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        # get name, arg from every tool_calls (ai-request)
        for tool_call in message.tool_calls:

            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f">> Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)  # string -> python obj
            except json.JSONDecodeError:
                arguments = {}

            # Get func
            function_to_run = FUNCTION_MAP.get(function_name)

            # Run func
            result = function_to_run(**arguments)
            print(
                f">> Ran {function_name} with args {arguments} for a result of {result}"
            )

            # to memory(result)
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()

    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

In [15]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        tools=TOOLS,  # Adding Tools
    )

    process_ai_response(response.choices[0].message)


while True:
    message = input("Input for LLM")  # 1. User send message

    if message == "quit" or message == "q":
        break
    else:
        messages.append(
            {"role": "user", "content": message}
        )  # 2. Add message to memory
        print(f"User: {message}")
        call_ai()  # 3. Call AI

User: My name is JM. Answer briefly.
AI: Nice to meet you, JM. How can I help?
User: What is my name?
AI: Your name is JM.
User: What's the weather in Jeju?
>> Calling function: get_weather with {"city":"Jeju"}
>> Ran get_weather with args {'city': 'Jeju'} for a result of 00 degrees celcius.
AI: Currently 0°C in Jeju. Want a forecast or more details?


In [ ]:
messages  # 2.6 Tool Results - 6:50

[{'role': 'user', 'content': 'My name is JM. Answer briefly.'},
 {'role': 'assistant', 'content': 'Nice to meet you, JM. How can I help?'},
 {'role': 'user', 'content': 'What is my name?'},
 {'role': 'assistant', 'content': 'Your name is JM.'},
 {'role': 'user', 'content': "What's the weather in Jeju?"},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_pxXpoRnBH5lLAYArSPY86b6V',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Jeju"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_pxXpoRnBH5lLAYArSPY86b6V',
  'name': 'get_weather',
  'content': '00 degrees celcius.'},
 {'role': 'assistant',
  'content': 'Currently 0°C in Jeju. Want a forecast or more details?'}]

In [ ]:
import json

# String
a = '{"city": "Jeju"}'
a[city]  # NameError: name 'city' is not defined

# Python Object(Dictionary)
b = json.loads(a)
b["city"]

'Jeju'

In [ ]:
get_weather(**b)  # str -> city='Jeju'
get_weather(city="Jeju")